## Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import langid
import openai
import time
import csv
import certifi

## Load the data

In [2]:
df = pd.read_csv('title_abstract_data_Large.csv', sep = ';')

## Data cleaning

In [3]:
df['abstractLength'] = df['abstract'].str.split().str.len()

In [4]:
df = df.loc[(df['abstractLength'] >= 170) & (df['abstractLength'] <= 230), ]

In [5]:
df = df.reset_index().drop(columns = ['index'])
df.head()

,title,pmid,url,abstract,abstractLength
0,Big Data and Data Science in Critical Care,29752973,https://pubmed.ncbi.nlm.nih.gov/29752973,The digitalization of the health-care system ...,178
1,The role of data science in healthcare advance...,34398394,https://pubmed.ncbi.nlm.nih.gov/34398394,Data science is an interdisciplinary field th...,209
2,Data science in cell imaging,33795377,https://pubmed.ncbi.nlm.nih.gov/33795377,Cell imaging has entered the 'Big Data' era. ...,209
3,Data science approaches to infectious disease ...,34802266,https://pubmed.ncbi.nlm.nih.gov/34802266,Novel data science approaches are needed to c...,204
4,Surgical data science - from concepts toward c...,34879287,https://pubmed.ncbi.nlm.nih.gov/34879287,Recent developments in data science in genera...,217


In [6]:
language = []
for i in df['abstract']:
    language.append(langid.classify(i)[0])

In [7]:
df['language'] = language

In [8]:
df = df.loc[df['language'] == 'en', ]

In [64]:
df = df[['title', 'abstract']].reset_index()

In [67]:
df.to_csv('writtenAbstracts.csv')

## Generate abstracts

In [19]:
openai.api_key = '#################################'

In [20]:
abstracts = []

In [21]:
for title in df['title']:
    completion = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = [
                {"role": "user", "content": "write an abstract for a scientific paper of 170 to 230 words on the title: " + title}
            ]
        )
    abstracts.append(completion.choices[0].message.content)
    time.sleep(2)

In [25]:
titles = df['title']
generatedAbstracts = pd.DataFrame(data = titles, columns = ['title'])

In [26]:
generatedAbstracts['abstracts'] = abstracts

In [60]:
generatedAbstracts.to_csv('generatedAbstracts.csv')